# Notebook 5 - Feature Importance

In this notebook, we check the feature importances using permutation.

In [1]:
# Global variables for testing changes to this notebook quickly
RANDOM_SEED = 0
NUM_FOLDS = 12

In [2]:
import numpy as np
import pandas as pd
import math
import scipy
import time
import pyarrow
import gc

# Model evaluation
from functools import partial
from sklearn.base import clone
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.inspection import partial_dependence, permutation_importance

# Plotting
import matplotlib
import seaborn as sns
from matplotlib import pyplot as plt

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [3]:
%%time

# Load original data
original = pd.read_feather('../data/original.feather')

# Label Encode
old_encoder = LabelEncoder()
original["Cover_Type"] = old_encoder.fit_transform(original["Cover_Type"])
y_train = original['Cover_Type'].iloc[:15119]
y_test = original['Cover_Type'].iloc[15119:]

Wall time: 51 ms


# Feature Engineering

In [4]:
def feature_engineering(data):
    df = data.copy()
    
    # Get columns
    shade_features = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']
    soil_features = [f'Soil_Type{i}' for i in range(1,41)]
    
    
    # Use float64 for calculations
    for col, dtype in df.dtypes.iteritems():
        if dtype.name.startswith('float'):
            df[col] = df[col].astype('float64')
    
    # Replace soil type columns with categoricals
    df['Soil_Type'] = 0
    for i in range(1,41):
        df['Soil_Type'] += i*df[f'Soil_Type{i}']
        
        
    df['Soil_12_32'] = df['Soil_Type32'] + df['Soil_Type12']
    df['Soil_Type23_22_32_33'] = df['Soil_Type23'] + df['Soil_Type22'] + df['Soil_Type32'] + df['Soil_Type33']
    #df['Aspect_360'] = df['Aspect'] % 360
    #df['Aspect_Sine'] = (df['Aspect']* np.pi / 180).apply(np.sin)
    #df['Aspect_Alt'] = (df['Aspect']-180).where(df['Aspect']+180 > 360, df['Aspect'] + 180)
    df['Horizontal_Distance_To_Roadways_Log'] = [math.log(v+1) for v in df['Horizontal_Distance_To_Roadways']]
    #df['Horizontal_Distance_To_Hydrology_Log'] = [math.log(v+1) for v in df['Horizontal_Distance_To_Hydrology']]
    #df['Elev_Binned'] = [math.floor(v/50.0) for v in df['Elevation']]
    #df["Hydro_Taxicab"] = np.abs(df["Horizontal_Distance_To_Hydrology"]) + np.abs(df["Vertical_Distance_To_Hydrology"])
    #df["Hydro_Euclid"] = (df["Horizontal_Distance_To_Hydrology"]**2 + np.abs(df["Vertical_Distance_To_Hydrology"])**2)**0.5
    #df['Water_Direction'] = df['Vertical_Distance_To_Hydrology'].apply(np.sign)
    df['Water Elevation'] = df['Elevation'] - df['Vertical_Distance_To_Hydrology']
    #df["Hillshade_Avg"] = df[shade_features].mean(axis=1)
    #df['Hillshade_Range'] = df[shade_features].max(axis=1) - df[shade_features].min(axis=1)
    #df['Hillshade'] = df[shade_features].sum(axis=1)
    df['Hydro_Fire_1'] = df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Fire_Points']
    df['Hydro_Fire_2'] = abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Fire_Points'])
    df['Hydro_Road_1'] = abs(df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Roadways'])
    df['Hydro_Road_2'] = abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Roadways'])
    df['Fire_Road_1'] = abs(df['Horizontal_Distance_To_Fire_Points'] + df['Horizontal_Distance_To_Roadways'])
    df['Fire_Road_2'] = abs(df['Horizontal_Distance_To_Fire_Points'] - df['Horizontal_Distance_To_Roadways'])
    #df['EHiElv'] = df['Horizontal_Distance_To_Roadways'] * df['Elevation']
    #df['EViElv'] = df['Vertical_Distance_To_Hydrology'] * df['Elevation']
    df['EVDtH'] = df.Elevation - df.Vertical_Distance_To_Hydrology
    #df['EHDtH'] = df.Elevation - df.Horizontal_Distance_To_Hydrology * 0.2
    df['Elev_3Horiz'] = df['Elevation'] + df['Horizontal_Distance_To_Roadways']  + df['Horizontal_Distance_To_Fire_Points'] + df['Horizontal_Distance_To_Hydrology']
    df['Elev_Road_1'] = df['Elevation'] + df['Horizontal_Distance_To_Roadways']
    df['Elev_Road_2'] = df['Elevation'] - df['Horizontal_Distance_To_Roadways']
    df['Elev_Fire_1'] = df['Elevation'] + df['Horizontal_Distance_To_Fire_Points']
    df['Elev_Fire_2'] = df['Elevation'] - df['Horizontal_Distance_To_Fire_Points']
    #df['Elev_Hillshade_1'] = df['Elevation'] - df['Hillshade']
    #df['Elev_Hillshade_2'] = df['Elevation'] + df['Hillshade']
    
    # ELU soil codes
    code = {
        1:2702,2:2703,3:2704,4:2705,5:2706,6:2717,7:3501,8:3502,9:4201,
        10:4703,11:4704,12:4744,13:4758,14:5101,15:5151,16:6101,17:6102,
        18:6731,19:7101,20:7102,21:7103,22:7201,23:7202,24:7700,25:7701,
        26:7702,27:7709,28:7710,29:7745,30:7746,31:7755,32:7756,33:7757,
        34:7790,35:8703,36:8707,37:8708,38:8771,39:8772,40:8776
    }
    
    # Climatic Zone
    df['Climatic_Zone'] = df['Soil_Type'].apply(lambda x: int(str(code[x])[0]))
    
    # Geologic Zone
    df['Geologic_Zone'] = df['Soil_Type'].apply(lambda x: int(str(code[x])[1]))
    
    # Surface Cover
    no_desc = [7,8,14,15,16,17,19,20,21,23,35]
    stony = [6,12]
    very_stony = [2,9,18,26]
    extremely_stony = [1,22,24,25,27,28,29,30,31,32,33,34,36,37,38,39,40]
    rubbly = [3,4,5,10,11,13]
    surface_cover = {i:0 for i in no_desc}
    surface_cover.update({i:1 for i in stony})
    surface_cover.update({i:2 for i in very_stony})
    surface_cover.update({i:3 for i in extremely_stony})
    surface_cover.update({i:4 for i in rubbly})
    
    df['Surface_Cover'] = df['Soil_Type'].apply(lambda x: surface_cover[x])

    # Rock Size
    no_desc = [7,8,14,15,16,17,19,20,21,23,35]
    stones = [1,2,6,9,12,18,24,25,26,27,28,29,30,31,32,33,34,36,37,38,39,40]
    boulders = [22]
    rubble = [3,4,5,10,11,13]
    rock_size = {i:0 for i in no_desc}
    rock_size.update({i:1 for i in stones})
    rock_size.update({i:2 for i in boulders})
    rock_size.update({i:3 for i in rubble})
    
    df['Rock_Size'] = df['Soil_Type'].apply(lambda x: rock_size[x])

            
    # Wilderness Interactions
    df['Climate_Area1'] = df['Wilderness_Area1']*df['Climatic_Zone'] 
    df['Climate_Area2'] = df['Wilderness_Area2']*df['Climatic_Zone'] 
    df['Climate_Area3'] = df['Wilderness_Area3']*df['Climatic_Zone'] 
    df['Climate_Area4'] = df['Wilderness_Area4']*df['Climatic_Zone'] 
    #df['Geologic_Area1'] = df['Wilderness_Area1']*df['Geologic_Zone'] 
    #df['Geologic_Area2'] = df['Wilderness_Area2']*df['Geologic_Zone']  
    #df['Geologic_Area3'] = df['Wilderness_Area3']*df['Geologic_Zone'] 
    #df['Geologic_Area4'] = df['Wilderness_Area4']*df['Geologic_Zone'] 
    df['Rock_Area1'] = df['Wilderness_Area1']*df['Rock_Size'] 
    df['Rock_Area2'] = df['Wilderness_Area2']*df['Rock_Size']   
    df['Rock_Area3'] = df['Wilderness_Area3']*df['Rock_Size']  
    df['Rock_Area4'] = df['Wilderness_Area4']*df['Rock_Size']
    df['Surface_Area1'] = df['Wilderness_Area1']*df['Surface_Cover'] 
    df['Surface_Area2'] = df['Wilderness_Area2']*df['Surface_Cover']   
    df['Surface_Area3'] = df['Wilderness_Area3']*df['Surface_Cover']  
    df['Surface_Area4'] = df['Wilderness_Area4']*df['Surface_Cover'] 
    df['Soil29_Area1'] = df['Soil_Type29'] + df['Wilderness_Area1']
    df['Soil3_Area4'] = df['Wilderness_Area4'] + df['Soil_Type3']
    
    # Drop redundant Soil columns
    df.drop(soil_features, axis = 1, inplace = True)
    
    # Fill NA
    df.fillna(0, inplace = True)
    
    # Downcast variables
    for col, dtype in df.dtypes.iteritems():
        if dtype.name.startswith('int'):
            df[col] = pd.to_numeric(df[col], downcast ='integer')
        elif dtype.name.startswith('float'):
            df[col] = pd.to_numeric(df[col], downcast ='float')
    
    return df

In [5]:
%%time

original = feature_engineering(original)

# Get feature columns
features = [x for x in original.columns if x not in ['Id','Cover_Type']]

Wall time: 1.92 s


# Training/Scoring Function

In [6]:
def train_original(sklearn_model):
    
    # Train/Test split
    X_temp = original[features].iloc[:15119]
    X_test = original[features].iloc[15119:]
    y_temp = original['Cover_Type'].iloc[:15119]
    y_test = original['Cover_Type'].iloc[15119:]
    
    # Store the out-of-fold predictions
    test_preds = np.zeros((X_test.shape[0],7))
    oof_preds = np.zeros((X_temp.shape[0],))
    fi_scores = np.zeros((X_temp.shape[1],))
    scores, times = np.zeros(NUM_FOLDS), np.zeros(NUM_FOLDS)
    
    # Stratified k-fold cross-validation
    skf = StratifiedKFold(n_splits = NUM_FOLDS, shuffle = True, random_state = RANDOM_SEED)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X_temp,y_temp)):
       
        # Training and Validation Sets
        X_train, X_valid = X_temp.iloc[train_idx], X_temp.iloc[valid_idx]
        y_train, y_valid = y_temp.iloc[train_idx], y_temp.iloc[valid_idx]
        
        # Create model
        start = time.time()
        model = clone(sklearn_model)
        model.fit(X_train, y_train)

        # Permutation Importance
        result = permutation_importance(
            model, X_valid, y_valid, 
            n_repeats=5, random_state=RANDOM_SEED, n_jobs=-1
        )
        fi_scores += result.importances_mean / NUM_FOLDS

        # validation/holdout predictions
        valid_preds = np.ravel(model.predict(X_valid))
        oof_preds[valid_idx] = valid_preds
        test_preds += model.predict_proba(X_test)

        # Save scores and times
        scores[fold] = accuracy_score(y_valid, valid_preds)
        end = time.time()
        times[fold] = end-start
        print(f'Fold {fold}: {round(scores[fold], 5)} in {round(times[fold], 2)}s')
        time.sleep(0.5)
    
    test_preds = np.argmax(test_preds, axis = 1)
    test_score = accuracy_score(y_test, test_preds)
    print('\n'+model.__class__.__name__)
    print("Train Accuracy:", round(scores.mean(), 5))
    print('Test Accuracy:', round(test_score, 5))
    print(f'Training Time: {round(times.sum(), 2)}s')
    
    return pd.Series(data = fi_scores, index = features).sort_values()

# Feature Importance

In [7]:
# Extremely Randomized Trees
extratrees = ExtraTreesClassifier(
    n_jobs = -1,
    random_state = RANDOM_SEED,
    max_features = None,
)

In [8]:
train_original(extratrees)

Fold 0: 0.90873 in 20.17s
Fold 1: 0.91508 in 15.65s
Fold 2: 0.9 in 15.11s
Fold 3: 0.90556 in 15.4s
Fold 4: 0.89921 in 16.01s
Fold 5: 0.91349 in 19.75s
Fold 6: 0.91111 in 19.59s
Fold 7: 0.90714 in 15.38s
Fold 8: 0.89921 in 15.16s
Fold 9: 0.91667 in 15.24s
Fold 10: 0.90794 in 15.51s
Fold 11: 0.91581 in 14.61s

ExtraTreesClassifier
Train Accuracy: 0.90833
Test Accuracy: 0.81348
Training Time: 197.58s


Climate_Area1                         -0.000238
Surface_Area4                          0.000027
Surface_Area2                          0.000079
Rock_Area3                             0.000093
Rock_Area2                             0.000093
Wilderness_Area1                       0.000119
Wilderness_Area3                       0.000172
Rock_Area4                             0.000172
Climate_Area4                          0.000371
Surface_Area3                          0.000463
Wilderness_Area2                       0.000503
Rock_Size                              0.000556
Climate_Area2                          0.000701
Surface_Area1                          0.000754
Rock_Area1                             0.000926
Soil_12_32                             0.001032
Climate_Area3                          0.001032
Surface_Cover                          0.001416
Horizontal_Distance_To_Roadways_Log    0.001455
Hillshade_3pm                          0.001482
Slope                                  0